![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

In [ ]:
!pip install -q pyspark==3.3.0 spark-nlp==4.2.0

In [ ]:
import sparknlp

spark = sparknlp.start()

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version 4.2.0
Apache Spark version: 3.3.0


In [ ]:
documentAssembler = DocumentAssembler() \
        .setInputCol("text") \
        .setOutputCol("documents")

t5 = T5Transformer.pretrained("t5_small_wikiSQL") \
        .setTask("translate English to SQL:") \
        .setInputCols(["documents"]) \
        .setMaxOutputLength(200) \
        .setOutputCol("sql")

pipeline = Pipeline().setStages([documentAssembler, t5])

t5_small_wikiSQL download started this may take some time.
Approximate size to download 249.9 MB
[OK!]


In [ ]:
data = spark.createDataFrame([["How many customers from Brazil have ordered more than 2 items on 2022-06-01?"]]).toDF("text")

result = pipeline.fit(data).transform(data)

result.select("sql.result").show(truncate=False)

+-----------------------------------------------------------------------------------------------+
|result                                                                                         |
+-----------------------------------------------------------------------------------------------+
|[SELECT COUNT Customers FROM table WHERE Orders > 2 AND Country = brazil AND Date = 2022-06-01]|
+-----------------------------------------------------------------------------------------------+

